# 05 - Despliegue y Monitorización en Azure ML

Este notebook demuestra el despliegue de modelos y configuración de monitorización.

**Objetivos:**
- Registrar modelo en Azure ML
- Crear managed online endpoint
- Configurar monitorización con Application Insights
- Detectar data drift
- Probar endpoint

In [ ]:
import json
import joblib
import pandas as pd
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    Model, ManagedOnlineEndpoint, ManagedOnlineDeployment,
    Environment, CodeConfiguration
)
from azure.identity import DefaultAzureCredential

print("✅ Imports completados")

## 1. Conexión a Azure ML

In [ ]:
# Configurar credenciales
credential = DefaultAzureCredential()

# Conectar a workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="<YOUR_SUBSCRIPTION_ID>",
    resource_group_name="<YOUR_RESOURCE_GROUP>",
    workspace_name="<YOUR_WORKSPACE>"
)

print(f"✅ Conectado a: {ml_client.workspace_name}")

## 2. Registro del Modelo

💡 **Copilot tip:** Usa versionado semántico y tags descriptivos para gestión.

In [ ]:
# Crear modelo registrado
model = Model(
    path="./models/best_model.joblib",
    name="clasificador-rf",
    description="Random Forest optimizado para clasificación binaria",
    tags={
        "framework": "scikit-learn",
        "algorithm": "RandomForest",
        "optimization": "optuna",
        "environment": "production"
    }
)

# Registrar
registered_model = ml_client.models.create_or_update(model)
print(f"✅ Modelo registrado:")
print(f"   Nombre: {registered_model.name}")
print(f"   Versión: {registered_model.version}")
print(f"   ID: {registered_model.id}")

## 3. Crear Script de Scoring

In [ ]:
%%writefile score.py
import json
import joblib
import numpy as np
import pandas as pd
import os
from azureml.core import Model

def init():
    """Inicializar modelo y scaler"""
    global model, scaler
    
    # Cargar modelo
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'best_model.joblib')
    model = joblib.load(model_path)
    
    # Cargar scaler (si existe)
    scaler_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'scaler.joblib')
    if os.path.exists(scaler_path):
        scaler = joblib.load(scaler_path)
    else:
        scaler = None
    
    print("Modelo cargado exitosamente")

def run(raw_data):
    """Ejecutar inferencia"""
    try:
        # Parsear input
        data = json.loads(raw_data)['data']
        df = pd.DataFrame(data)
        
        # Aplicar scaler si existe
        if scaler is not None:
            df = pd.DataFrame(scaler.transform(df), columns=df.columns)
        
        # Predecir
        predictions = model.predict(df)
        probabilities = model.predict_proba(df)
        
        # Formatear resultado
        result = {
            'predictions': predictions.tolist(),
            'probabilities': probabilities.tolist()
        }
        
        return json.dumps(result)
    
    except Exception as e:
        error = str(e)
        return json.dumps({"error": error})

## 4. Crear Endpoint

In [ ]:
import time

# Definir endpoint
endpoint_name = f"clasificador-ep-{int(time.time())}"

endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="Endpoint para clasificación binaria",
    auth_mode="key",
    tags={"model": "random-forest", "version": "1.0"}
)

# Crear endpoint
print(f"🚀 Creando endpoint: {endpoint_name}...")
ml_client.online_endpoints.begin_create_or_update(endpoint).result()
print("✅ Endpoint creado")

## 5. Crear Deployment

In [ ]:
# Definir deployment
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=registered_model.id,
    code_configuration=CodeConfiguration(
        code=".",
        scoring_script="score.py"
    ),
    environment="azureml:inference-env@latest",
    instance_type="Standard_DS2_v2",
    instance_count=1
)

# Crear deployment
print("🚀 Creando deployment...")
ml_client.online_deployments.begin_create_or_update(blue_deployment).result()
print("✅ Deployment creado")

# Asignar 100% del tráfico
endpoint.traffic = {"blue": 100}
ml_client.online_endpoints.begin_create_or_update(endpoint).result()
print("✅ Tráfico configurado: 100% a 'blue'")

## 6. Probar Endpoint

In [ ]:
# Cargar datos de prueba
X_test = pd.read_parquet('./data/X_test.parquet')
sample_data = X_test.head(5)

# Preparar request
request_data = {
    "data": sample_data.to_dict(orient='records')
}

# Invocar endpoint
print("🧪 Probando endpoint...")
response = ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    request_file=json.dumps(request_data)
)

print("\n✅ Respuesta del endpoint:")
result = json.loads(response)
print(json.dumps(result, indent=2))

## 7. Configurar Monitorización

💡 **Copilot tip:** Usa Application Insights para telemetría en tiempo real.

In [ ]:
# Obtener endpoint details
endpoint_details = ml_client.online_endpoints.get(endpoint_name)

print("📊 Configuración de Monitorización:")
print(f"   Endpoint: {endpoint_details.name}")
print(f"   Scoring URI: {endpoint_details.scoring_uri}")
print(f"   Estado: {endpoint_details.provisioning_state}")
print(f"\n💡 Configura alertas en Application Insights para:")
print("   - Latencia > 1000ms")
print("   - Error rate > 5%")
print("   - Data drift detectado")

## 8. Simulación de Data Drift

Demostración de cómo detectar drift usando Evidently.

In [ ]:
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset

# Cargar datos de entrenamiento y test
X_train = pd.read_parquet('./data/X_train.parquet')
X_test = pd.read_parquet('./data/X_test.parquet')

# Crear reporte de drift
drift_report = Report(metrics=[DataDriftPreset()])
drift_report.run(reference_data=X_train, current_data=X_test)

# Guardar reporte
drift_report.save_html('./reports/drift_report.html')
print("✅ Reporte de drift guardado en: ./reports/drift_report.html")

## 9. Cleanup (Opcional)

In [ ]:
# Descomentar para eliminar endpoint
# ml_client.online_endpoints.begin_delete(name=endpoint_name).result()
# print(f"🗑️  Endpoint {endpoint_name} eliminado")

## 10. Resumen

**Pasos completados:**
1. ✅ Modelo registrado en Azure ML
2. ✅ Managed online endpoint creado
3. ✅ Deployment con blue/green strategy
4. ✅ Endpoint probado exitosamente
5. ✅ Monitorización configurada
6. ✅ Data drift monitoring habilitado

**Próximos pasos:**
- Configurar CI/CD pipeline para despliegues automáticos
- Implementar A/B testing
- Configurar reentrenamiento automático al detectar drift